In [1]:
# import pandas library terlebih dahulu untuk memuat dataset csv yang sudah disediakan
import pandas as pd

# baca file csv dan tampung kedalam variable
dataset = pd.read_csv('./UTS/data_survei_pasar.csv')
# coba untuk tampilkan terlebih dahulu
dataset

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [4]:
# import beberapa libary sklearn seperti SelectKBest, dan chi2 yang digunakan untuk melihat seberapa kuat hubungan pada data, metode univariate
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# buat attribut inputnya
attr_input = dataset.iloc[:,:20] # iloc digunakan untuk mengambil dengan range tertentu, iloc[row,column], iloc[start:finish, start:finish], (:) that means get all data row/column
# buat attribut outputnya
attr_output = dataset.iloc[:,-1] # minus (-) digunakan untuk mengambil data dari belakang, -1 = mengambil data terakhir

# buat object feature_selection
best_features = SelectKBest(score_func=chi2,k=10) # k adalah degree of freedom atau data/column yang diijinkan berubah untuk bervariasi tanpa merubah semuanya
# lakukan proses feature_selection
fit_results = best_features.fit(attr_input,attr_output)

# buat frame scorenya
data_scores = pd.DataFrame(fit_results.scores_)
# buat variable untuk menampung nama column yang terseleksi
data_columns = pd.DataFrame(attr_input.columns)

# gabungkan 2 frame tersebut
featureScore = pd.concat([data_columns,data_scores],axis=1) # axis 1 = columns, axis 0 = index
# buat/kasih nama labelnya
featureScore.columns = ['Specs','Score']

#filter 10 terbesar dari score
featureScore.nlargest(10, 'Score')

,Specs,Score
13,ram,931267.519053
11,px_height,17363.569536
0,battery_power,14129.866576
12,px_width,9810.586750
8,mobile_wt,95.972863
6,int_memory,89.839124
15,sc_w,16.480319
16,talk_time,13.236400
4,fc,10.135166
14,sc_h,9.614878


In [ ]:
# 10 Data Yang paling mempengaruhi harga dari sebuuah ponsel sudah tertampil

In [5]:
from sklearn import linear_model

columnMost4 = featureScore.nlargest(4, 'Score').Specs

x = dataset[columnMost4]
y = dataset[['price_range']]

regr = linear_model.LinearRegression()
regr.fit(x,y)

predictPrice = regr.predict([[16000,1920,1080,2048]])
predictPrice

array([[15.13919547]])

In [7]:
newDataset = pd.concat([x,y],axis=1)
newDataset.describe()

,ram,px_height,battery_power,px_width,price_range
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,2124.213000,645.108000,1238.518500,1251.515500,1.500000
std,1084.732044,443.780811,439.418206,432.199447,1.118314
min,256.000000,0.000000,501.000000,500.000000,0.000000
25%,1207.500000,282.750000,851.750000,874.750000,0.750000
50%,2146.500000,564.000000,1226.000000,1247.000000,1.500000
75%,3064.500000,947.250000,1615.250000,1633.000000,2.250000
max,3998.000000,1960.000000,1998.000000,1998.000000,3.000000


In [8]:
newDataset

# hapus baris yang terdapapat nilai 0
for row in newDataset.columns:
    newDataset = newDataset[newDataset[row] != 0]

newDataset.describe()

,ram,px_height,battery_power,px_width,price_range
count,1498.000000,1498.000000,1498.000000,1498.000000,1498.000000
mean,2570.172897,682.251001,1278.941923,1284.989987,2.000000
std,853.188835,459.094346,441.243159,433.161249,0.816497
min,387.000000,2.000000,501.000000,500.000000,1.000000
25%,1891.250000,300.750000,894.000000,899.750000,1.000000
50%,2573.500000,609.000000,1310.000000,1289.000000,2.000000
75%,3299.750000,991.750000,1663.750000,1670.750000,3.000000
max,3998.000000,1960.000000,1998.000000,1998.000000,3.000000


In [9]:
# cek apakah masih ada yang null
newDataset.isnull().sum()

ram              0
px_height        0
battery_power    0
px_width         0
price_range      0
dtype: int64

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# menampilkan plot table 
newDataset.corr().style.background_gradient().set_precision(2)

,ram,px_height,battery_power,px_width,price_range
ram,1.00,-0.17,-0.15,-0.13,0.85
px_height,-0.17,1.00,0.01,0.53,0.07
battery_power,-0.15,0.01,1.00,-0.02,0.14
px_width,-0.13,0.53,-0.02,1.00,0.11
price_range,0.85,0.07,0.14,0.11,1.00


In [11]:
x1 = newDataset.drop(columns='price_range') # mengambil data selain column price range
y1 = newDataset['price_range'] # mengambil column price_range

#Kedua, split data menjadi training and testing dengan porsi tertentu.
x_train, x_test, y_train, y_test = train_test_split(x1, y1, test_size=0.2, random_state=4)

# buat object regresi
lin_reg = LinearRegression()

# training the model menggunakan training data yang sudah displit sebelumnya.
lin_reg.fit(x_train, y_train)

# cari tau nilai slope/koefisien (m) dan intercept (b).
print(lin_reg.coef_)
print(lin_reg.intercept_)

[0.0008866  0.00024424 0.00052847 0.00030397]
-1.5180106383505605


In [12]:
lin_reg.score(x_test,y_test)

0.8479201075974119

In [13]:
# kita cari tahu accuracy score dari model kita menggunakan testing data yang sudah displit sebelumnya.
# [ram, px_height, battery_power, px_width]
print("price_range\t:", str(lin_reg.predict([[16000,1920,1080,2048]])))

price_range	: [14.32976241]
